In [148]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T

from sklearn import datasets
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.datasets import load_diabetes

# df = pd.read_csv('tabular_data/tabular_data/clean_tabular_data.csv')
# X = df[["guests","beds","bathrooms","Cleanliness_rating","Accuracy_rating","Communication_rating","Location_rating","Check-in_rating","Value_rating","amenities_count"]] # get all columns except the last one
# y = df['Price_Night']  # get the 'Price_Night' column

X,y = load_diabetes(return_X_y=True,as_frame=True)
X = X.to_numpy()
y = y.to_numpy()
X= X.astype('float32')
y = y.astype('float32')
y = y.reshape(-1,1)

X_scaler = MinMaxScaler()
X_scaled = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y_scaled = y_scaler.fit_transform(y)
y_scaled = y_scaled.ravel()

# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.33, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.33, random_state=42)

class TransformFn:
    def __init__(self,k):
        self.top_k_scores_list = []
        self.top_k_names_list = []
        self.k = k

    def select_features(self, features, labels):
        k = self.k
        selector = SelectKBest(k=k, score_func=f_regression)
        selected_features = selector.fit_transform(features, labels)
        scores = selector.scores_
        if self.k != 'all':
            top_k_indices = np.argpartition(scores, -k)[-k:]
            top_k_scores = scores[top_k_indices]
            top_k_names = top_k_indices
            self.top_k_scores_list = top_k_scores
            self.top_k_names_list = top_k_names
            
        return selected_features, labels

transform_fn1 = TransformFn(k=3)

def to_tensor(sample):
     return torch.from_numpy(sample)

class Data(Dataset):

  def __init__(self, X: np.ndarray, y: np.ndarray) -> None:

    self.X = torch.from_numpy(X.astype(np.float32))
    self.y = torch.from_numpy(y.astype(np.float32))
    self.X = transform_fn1.select_features(X, y)[0]
    self.y = transform_fn1.select_features(X, y)[1]
    self.X = to_tensor(self.X)
    self.y = to_tensor(self.y)

  def __getitem__(self, index: int) -> tuple:

    return self.X[index], self.y[index]

  def __len__(self) -> int:
    return self.X.shape[0]

batch_size = 100

# Generate the trainingand testing datasets
train_dataset = Data(X_train, y_train)
test_dataset = Data(X_test, y_test)

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  

In [151]:
import xgboost as xgb
from xgboost import plot_importance
xgb_reg_start = time.time()
xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)
training_preds_xgb_reg = xgb_reg.predict(X_train)
val_preds_xgb_reg = xgb_reg.predict(X_test)


xgb_reg_end = time.time()

print(f"Time taken to run: {round((xgb_reg_end - xgb_reg_start)/60,1)} minutes")
print("\nTraining MSE:", round(mean_squared_error(y_train, training_preds_xgb_reg),4))
print("Validation MSE:", round(mean_squared_error(y_test, val_preds_xgb_reg),4))
print("\nTraining r2:", round(r2_score(y_train, training_preds_xgb_reg),4))
print("Validation r2:", round(r2_score(y_test, val_preds_xgb_reg),4))
# ft_weights_xgb_reg = pd.DataFrame(xgb_reg.feature_importances_, columns=['weight'], index=X_train.columns)
# ft_weights_xgb_reg.sort_values('weight', inplace=True)
# ft_weights_xgb_reg
# # Plotting feature importances
# plt.figure(figsize=(8,20))
# plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg.weight, align='center') 
# plt.title("Feature importances in the XGBoost model", fontsize=14)
# plt.xlabel("Feature importance")
# plt.margins(y=0.01)
# plt.show()

Time taken to run: 0.0 minutes

Training MSE: 0.0
Validation MSE: 0.0383

Training r2: 1.0
Validation r2: 0.3142


In [152]:
class NeuralNetwork(nn.Module):
  '''Linear Regression Model'''

  def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:
    '''The network has 4 layers
         - input layer
         - hidden layer
         - hidden layer
         - output layer'''
    super(NeuralNetwork, self).__init__()
    self.input_to_hidden = nn.Linear(input_dim, hidden_dim)
    self.hidden_layer_1 = nn.Linear(hidden_dim, hidden_dim)
    self.hidden_activation_layer = nn.LeakyReLU(0.3)
    self.hidden_layer_2 = nn.Linear(hidden_dim, hidden_dim)
    self.hidden_to_output = nn.Linear(hidden_dim, output_dim)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # no activation and no softmax at the end
    x = self.input_to_hidden(x)
    x = self.hidden_layer_1(x)
    x = self.hidden_activation_layer(x)
    x = self.hidden_layer_2(x)
    x = self.hidden_to_output(x)
    return x


input_dim = transform_fn1.k
output_dim = 1
# number of hidden layers
hidden_layers =3

model = NeuralNetwork(input_dim, hidden_layers,output_dim)
learning_rate = 0.0005

print(input_dim)

# criterion to computes the loss between input and target
criterion = nn.MSELoss()

# optimizer that will be used to update weights and biases
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005)

# start training

writer = SummaryWriter()

epochs = 100000
for epoch in range(epochs):
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(train_data_loader):
    inputs = inputs.view(-1, input_dim)
    labels = labels.view(-1, output_dim)
    # forward propagation
    outputs = model(inputs)
    # loss = criterion(outputs, labels)

    loss = F.mse_loss(outputs,labels)
    
    # set optimizer to zero grad to remove previous epoch gradients
    optimizer.zero_grad()

    # backward propagation
    loss.backward()

    # optimize
    optimizer.step()


    outputs_detached = outputs.detach().numpy()
    labels_detached = labels.detach().numpy()
    loss = loss.detach().numpy()

    outputs_unscaled = y_scaler.inverse_transform(outputs_detached.reshape(-1, 1))
    labels_unscaled = y_scaler.inverse_transform(labels_detached.reshape(-1, 1))
    loss_unscaled = y_scaler.inverse_transform(loss.reshape(-1, 1))
    
    r2 = r2_score(labels_unscaled,outputs_unscaled)

    running_loss += loss

    # running_loss += loss.item()
  writer.add_scalar('Loss/train', running_loss, epoch)
  writer.add_scalar('R2/train', r2, epoch)

  # display statistics
  loss_cal = running_loss/ (i + 1)
  loss_str= np.format_float_positional(loss_cal, precision=2)
  if not ((epoch + 1) % (epochs // 10)):
    print(f'Epochs:{epoch + 1:5d} | ' \
          f'Batches per epoch: {i + 1:3d} | ' \
          f'Loss: {loss_str} | ' \
          f'R2: {r2}')

writer.close()
# testdata = Data(X_test, y_test)
# testloader = DataLoader(testdata, batch_size=batch_size, 
#                         shuffle=True)
# Validate trained model using the test dataset
model.eval()
with torch.no_grad():
  loss = 0
  for i, (inputs, labels) in enumerate(test_data_loader):
    # calculate output by running through the network
    inputs = inputs.view(-1, input_dim)
    labels = labels.view(-1, output_dim)
    predictions = model(inputs)

    predictions_detached = predictions.detach().numpy()
    labels_detached = labels.detach().numpy()

    y_test_unscaled = y_scaler.inverse_transform(labels_detached.reshape(-1, 1))
    predictions_unscaled = y_scaler.inverse_transform(predictions_detached.reshape(-1, 1))

    loss += F.mse_loss(predictions, labels)
    r2 = r2_score(y_test_unscaled,predictions_unscaled)
  
  mse = loss / (i + 1)
  rmse = np.sqrt(mse)
  print(f"The r2 value is: {r2:.3f}")
  print(f'MSE Loss: {mse:.5f}')
  print(f'RMSE Loss: {rmse:.5f}')

3
Epochs:10000 | Batches per epoch:   3 | Loss: 0.05 | R2: 0.06727693635864562
Epochs:20000 | Batches per epoch:   3 | Loss: 0.05 | R2: 0.21877801285531662
Epochs:30000 | Batches per epoch:   3 | Loss: 0.03 | R2: 0.3884077927212519
Epochs:40000 | Batches per epoch:   3 | Loss: 0.03 | R2: 0.5070038533800001
Epochs:50000 | Batches per epoch:   3 | Loss: 0.03 | R2: 0.5379181232749511
Epochs:60000 | Batches per epoch:   3 | Loss: 0.03 | R2: 0.4690031344830261
Epochs:70000 | Batches per epoch:   3 | Loss: 0.03 | R2: 0.3046447973632038
Epochs:80000 | Batches per epoch:   3 | Loss: 0.03 | R2: 0.48097066781493036
Epochs:90000 | Batches per epoch:   3 | Loss: 0.03 | R2: 0.41247239382154943
Epochs:100000 | Batches per epoch:   3 | Loss: 0.03 | R2: 0.5162677394616744
The r2 value is: 0.489
MSE Loss: 0.03035
RMSE Loss: 0.17420
